In [1]:
import numpy as np
import cv2

### Helper Functions

In [2]:
def partition_image(img_masked, num_partitions = 4):
    
    zero_row = np.zeros(img_masked.shape[1:])
    partitions = []
    cut_start_index = 0
    should_cut = False
    
    for i, row in enumerate(img_masked):
        row_is_zero = np.all(np.equal(row, zero_row))
        if not should_cut and not row_is_zero:
            should_cut = True
        elif should_cut and row_is_zero:
            partitions.append(img_masked[cut_start_index : i + 1])
            cut_start_index = i + 1
            should_cut = False
            if len(partitions) == num_partitions:
                return partitions
    print("Warning: Didn't reach full number of partitions {}, \
          returning {} partitions instead".format(num_partitions, len(partitions)))
    return partitions

In [3]:
def compute_avg_colored_pixel(partition):
    
    zero_row = np.zeros(partition.shape[2], dtype='int64')
    pixels = partition.reshape(-1, partition.shape[-1])
    non_black_pixel_indices = np.where(np.any(pixels != zero_row, axis=1))
    non_black_pixels = pixels[non_black_pixel_indices]
    return np.average(non_black_pixels, axis=0)

### Retrieving Images

In [4]:
import glob
masked_images = []
Y = []
for img_path in glob.glob('masked_data/*.jpg'):
    masked_images.append(cv2.imread(img_path))
    y = img_path[12:-7].replace('p', '.')
    Y.append(y)
Y = np.array(Y).astype('float')[:, np.newaxis] # makes shape (90, 1)
print(Y.shape)

(90, 1)


### Parsing Colors from Images

In [5]:
X = []
for masked_image in masked_images:
    partitions = partition_image(masked_image)
    avg_pixels = [compute_avg_colored_pixel(partition) for partition in partitions]
    X.append(np.concatenate(avg_pixels).ravel()) # converts list of arrays to one array
X = np.asmatrix(X)
print(X.shape)

(90, 12)


### Saving Parsed Color Data

In [6]:
np.save('X.npy', X)
np.save('Y.npy', Y)